# Download Libraries & Ollama

In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [2]:
!pip install -qU pinecone-client pinecone-datasets langchain-pinecone

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.9/34.9 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 51.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.3/291.3 kB 20.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.2/115.2 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.4/73.4 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 47.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.8/143.8 kB 14.4 MB/s eta 0:00:00


In [3]:
!pip install --quiet langchain_community tiktoken langchainhub langchain langgraph langchain-mistralai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.7/817.7 kB 33.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.3/59.3 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.1/121.1 kB 14.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
botocore 1.31.17 requires urllib3<1.27,>=1.25.4, but you have urllib3 2.2.1 which is incompatible.


In [4]:
%pip install --upgrade --quiet  wikipedia

  Preparing metadata (setup.py) ... done


# Loading Libraries

In [113]:
import os
import bs4
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_community.chat_models import ChatOllama
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
import pinecone
from pinecone import Pinecone, ServerlessSpec, PodSpec
from langchain_community.document_loaders import WikipediaLoader
from google.colab import userdata
import time
from langchain import hub
from langchain_community.embeddings import OllamaEmbeddings
from langchain.docstore.document import Document
from langchain_community.vectorstores import Chroma
from langchain.utils.math import cosine_similarity

# Setting up langsmith & Pinecone APIs

In [86]:
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
# In the following two lines either replace the function with
# your API keys or save them in the secrets section in google
# colab
os.environ['LANGCHAIN_API_KEY'] = userdata.get('LANGCHAIN_API_KEY')
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')
pinecone_api_key = os.environ['PINECONE_API_KEY']
use_serverless = True

# Loading Docs

In [87]:
aero_docs = WikipediaLoader(query="Aeronautics", lang="en", load_max_docs=1).load()
aero_docs.extend(WikipediaLoader(query="Aerospace engineering", lang="en", load_max_docs=1).load())
aero_docs.extend(WikipediaLoader(query="Avionics", lang="en", load_max_docs=1).load())
aero_docs.extend(WikipediaLoader(query="Flight dynamics", lang="en", load_max_docs=1).load())
aero_docs.extend(WikipediaLoader(query="Aircraft design process", lang="en", load_max_docs=1).load())
aero_docs.extend(WikipediaLoader(query="Aircraft flight control system", lang="en", load_max_docs=1).load())
aero_docs.extend(WikipediaLoader(query="Aircraft flight mechanics", lang="en", load_max_docs=1).load())

In [88]:
naval_docs = WikipediaLoader(query="Naval architecture", lang="en", load_max_docs=1).load()
naval_docs.extend(WikipediaLoader(query="Ship stability", lang="en", load_max_docs=1).load())
naval_docs.extend(WikipediaLoader(query="Hydraulic engineering", lang="en", load_max_docs=1).load())
naval_docs.extend(WikipediaLoader(query="Flight dynamics", lang="en", load_max_docs=1).load())
naval_docs.extend(WikipediaLoader(query="Fluid dynamics", lang="en", load_max_docs=1).load())
naval_docs.extend(WikipediaLoader(query="Hydrostatics", lang="en", load_max_docs=1).load())
naval_docs.extend(WikipediaLoader(query="Marine engineering", lang="en", load_max_docs=1).load())

# Document Chunking

In [89]:
def doc_chunk(docs, chunk_size= 5000, chunk_overlap= 500):
  # Split
  text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
      chunk_size = chunk_size,
      chunk_overlap = chunk_overlap)

  # Make splits
  splits = text_splitter.split_documents(docs)
  return splits

In [90]:
summary_naval_splits = doc_chunk(naval_docs)
summary_aero_splits = doc_chunk(aero_docs)

In [91]:
retrieval_naval_splits = doc_chunk(naval_docs, chunk_size=500, chunk_overlap=50)
retrieval_aero_splits = doc_chunk(aero_docs, chunk_size=500, chunk_overlap=50)

In [92]:
type(summary_naval_splits[0])

langchain_core.documents.base.Document

# Summarizing documents

In [93]:
def get_split_summary(split, llm):
  prompt = """
  Give me a summary of the following document in 500 tokens:
  {document}
  """
  prompt_template = PromptTemplate(
      input_variables = ["document"],
      template = prompt
  )

  summarize_chain = prompt_template | llm | StrOutputParser()
  summary = summarize_chain.invoke({"document": split.page_content})
  summary = ":".join(summary.split(":")[1:])
  summary_doc = Document(page_content=summary, metadata=split.metadata)
  return summary_doc

In [94]:
summary_local_llm = "llama3"

summary_llm = ChatOllama(model=summary_local_llm, temperature=0.75)

In [95]:
def summarize_all_splits(summary_naval_splits, summary_aero_splits, llm):
  aero_summary_docs = []
  naval_summary_docs = []
  for split in summary_naval_splits:
    summary_doc = get_split_summary(split, llm)
    naval_summary_docs.append(summary_doc)
  for split in summary_aero_splits:
    summary_doc = get_split_summary(split, llm)
    aero_summary_docs.append(summary_doc)
  return aero_summary_docs, naval_summary_docs

In [96]:
aero_summary_docs, naval_summary_docs = summarize_all_splits(summary_naval_splits, summary_aero_splits, summary_llm)

In [97]:
def summarize_all_docs(docs, llm):
  combined_doc = "\n".join([doc.page_content for doc in docs])
  prompt = """
  Give me a summary of the following document in 500 tokens:
  # {combined_doc}
  """
  prompt_template = PromptTemplate(
      input_variables = ["combined_doc"],
      template = prompt
  )

  summarize_chain = prompt_template | llm | StrOutputParser()
  summary = summarize_chain.invoke({"combined_doc": combined_doc})
  summary = ":".join(summary.split(":")[1:])
  summary_doc = Document(page_content=summary)
  return summary_doc

In [98]:
aero_summary = summarize_all_docs(aero_summary_docs, summary_llm)
naval_summary = summarize_all_docs(naval_summary_docs, summary_llm)

In [99]:
type(aero_summary)

langchain_core.documents.base.Document

# Adding summaries to the chunks

In [100]:
retrieval_naval_splits = retrieval_naval_splits + naval_summary_docs
retrieval_naval_splits.append(naval_summary)
retrieval_aero_splits = retrieval_aero_splits + aero_summary_docs
retrieval_aero_splits.append(aero_summary)

# Loading the embeddings

In [101]:
embeddings = OllamaEmbeddings(model="mxbai-embed-large")

In [107]:
len(embeddings.embed_query("hello world"))

1024

# Creating Pinecone dbs

In [109]:
pc = Pinecone(api_key=pinecone_api_key)

def create_pinecone_index(pc, index_name, embedding_dim = 1024, use_serverless = True, metric = "dotproduct"):
    if use_serverless:
        spec = ServerlessSpec(cloud='aws', region='us-west-2')
    else:
        # if not using a starter index, you should specify a pod_type too
        spec = PodSpec()
    # check for and delete index if already exists
    if index_name in pc.list_indexes().names():
        pc.delete_index(index_name)
    # create a new index
    pc.create_index(
        index_name,
        dimension= embedding_dim,  # dimensionality of the embedding model
        metric= metric,
        spec=spec
    )
    # wait for index to be initialized
    while not pc.describe_index(index_name).status['ready']:
        time.sleep(1)

In [110]:
create_pinecone_index(pc, "aero-db")
create_pinecone_index(pc, "naval-db")

In [111]:
from langchain.vectorstores import Pinecone
def create_pinecone_vectorstore(splits, embeddings, index_name):
  vectorstore = Pinecone.from_documents(splits, embeddings, index_name = index_name)
  retriever = vectorstore.as_retriever()
  return vectorstore, retriever

In [112]:
aero_vectorstore, aero_retriever = create_pinecone_vectorstore(retrieval_aero_splits, embeddings, "aero-db")
naval_vectorstore, naval_retriever = create_pinecone_vectorstore(retrieval_naval_splits, embeddings, "naval-db")

# Semantinc Router

In [114]:
summaries = [aero_summary, naval_summary]
summary_names = ["aero", "naval"]
summary_embeddings = embeddings.embed_documents(summaries)

In [115]:
def query_router(query):
    # Embed question
    query_embedding = embeddings.embed_query(query)
    # Compute similarity
    similarity = cosine_similarity([query_embedding], summary_embeddings)[0]
    most_similar = summary_names[similarity.argmax()]
    return most_similar

def query_retriever(query, most_similar):
    if most_similar == "aero":
        return aero_retriever.get_relevant_documents(query)
    else:
        return naval_retriever.get_relevant_documents(query)

# RAG with LLaMA3

In [124]:
question = "What are the fundamental concept of fluid mechanics"
route = query_router(question)
retrieved_docs = query_retriever(question, route)
prompt = hub.pull("rlm/rag-prompt")
rag_chain = prompt | summary_llm | StrOutputParser()
result = rag_chain.invoke({"context": retrieved_docs, "question": question})

In [125]:
result

'The foundational axioms of fluid dynamics are the conservation laws specifically'

# RAG with Mistral

In [128]:
local_llm = "mistral:latest"

llm = ChatOllama(model=local_llm, temperature=0.75)

In [130]:
question = "What are the fundamental concept of fluid mechanics"
route = query_router(question)
retrieved_docs = query_retriever(question, route)
prompt = hub.pull("rlm/rag-prompt")
rag_chain = prompt | llm | StrOutputParser()
result = rag_chain.invoke({"context": retrieved_docs, "question": question})

In [131]:
result

' Fluid dynamics is a subdiscipline of fluid mechanics that describes the flow of liquids and gases. It has several applications, including calculating forces and moments on aircraft, determining mass flow rates in pipelines, predicting weather patterns, understanding nebulae in interstellar space, and modeling fission weapon detonation. The foundational axioms of fluid dynamics are the conservation laws for mass, linear momentum, and energy. Fluids are assumed to obey the continuum assumption, which assumes fluids are continuous rather than discrete. For Newtonian fluids, the momentum equations are described by the Navier-Stokes equations, which do not have a general closed-form solution and are primarily used in computational fluid dynamics. A thermodynamic equation of state is also required to describe the problem completely. For example, the perfect gas equation of state relates pressure, density, temperature, molar mass, and gas constant. Additionally, a constitutive relation may 